In [ ]:
# Preprocess Retriever data
# Very ineffective but works for now

In [344]:
import pandas as pd

In [304]:
# Input file is a txt file from Retriever 
# Only includes 500 texts with semi-structured metadata
# Here, we perform the pre-processing to extract the headlines and the page numbers

content = open("retriever_examples.txt").readlines()

In [301]:
def merge(sentence):
    # Concatenate list items into one string
    string = ''
    for i in sentence:
        if i[0].islower():
            string += " " + i.strip() # If start of list item is lowercase, join on whitespace
        else:
            string += ". " + i.strip() # Id start of list item is not lowercase, join on ". "
    return string.strip(". ")

In [305]:
# Lookup is the anchor
lookup = "Publicerat i print."
line_indices = [line_num for line_num, line_content in enumerate(content) if lookup in line_content]

In [345]:
potential_headlines = [] # List with some strings that look like headlines and pagenumbers

for line_index in line_indices:
    potential_headline = []
    candidate_range = [7, 6, 5, 4, 3, 2, 1]
    forbidden_strings = ["Nyheter:", "Aftonbladet, 2", "Datum 2", "===", "Dagens Nyheter, 2", "Expressen, 2", "Läs hela artikeln på", "Svenska Dagbladet, 2"] # Discard these lines, uninteresting for now. Revisit if more metadata is required.
    for candidate in candidate_range:
        if not any(content[line_index-candidate].startswith(s) for s in forbidden_strings):
            if content[line_index-candidate] != "\n":
                if content[line_index-candidate]:
                    headline = content[line_index-candidate]
                    potential_headline.append(headline.strip())                    
    potential_headlines.append(potential_headline) # Potential headlines are often splitted into multiple list items depending on their position in the original file

In [348]:
headlines = []

for headline in potential_headlines:
    h = headline[:-1] # Discard page number for now
    if len(h) > 1: # If headline more than one list item
        head = merge(h)
    else:
        head = " ".join(h) # Accept string as headline
    head = " ".join(h)
    page = head[-1].strip("Sida ").split(",")[0] # Sometimes there are multiple pagenumbers if the article spans more than one page. Choose the first one.
    headlines.append({'headline': head,
                      'page': page})
    
df = pd.DataFrame(headlines) # Create dataframe

In [349]:
df.to_csv("filtered_headlines.csv")